In [1]:
#neature network
import os
import pickle as pk
import numpy as np

#数据读取
def data_read(data_path):
    with open(data_path, 'rb') as dataSet:
        data_dict = pk.load(dataSet, encoding = 'latin1')
        #print(data_dict)
        xdata = data_dict['data']
        ydata = data_dict['labels']
           
        X_data = xdata.reshape(10000, 3*32*32)
        Y_label = np.array(ydata)
        dataSet.close()
    
        return X_data, Y_label
 
UbuntuBaseDir = '/media/cui/AC86F58E86F55972'
WinBaseDir = 'E:'
BaseDir = UbuntuBaseDir
#分别读取五部分的数据做价差验证，然后取平均值，得到最好的K和距离范式后再跑测试集
datadir1 = os.path.join(BaseDir, 'seedclass/机器学习/cs231/cifar-10-batches-py/data_batch_1')
datadir2 = os.path.join(BaseDir, 'seedclass/机器学习/cs231/cifar-10-batches-py/data_batch_2')
datadir3 = os.path.join(BaseDir, 'seedclass/机器学习/cs231/cifar-10-batches-py/data_batch_3')
datadir4 = os.path.join(BaseDir, 'seedclass/机器学习/cs231/cifar-10-batches-py/data_batch_4')
datadir5 = os.path.join(BaseDir, 'seedclass/机器学习/cs231/cifar-10-batches-py/data_batch_5')

xtr1, ylab1 = data_read(datadir1)
xtr2, ylab2 = data_read(datadir2)
xtr3, ylab3 = data_read(datadir3)
xtr4, ylab4 = data_read(datadir4)
xtr5, ylab5 = data_read(datadir5)
print('xtr1.shape',xtr1.shape)
print('ylab1.shape',ylab1.shape)

xtr1.shape (10000, 3072)
ylab1.shape (10000,)


In [2]:
import tensorflow as tf

def add_layer(inputs, in_size, out_size, activation_function=None): 
# add one more layer and return the output of this layer 
    Weights = tf.Variable(tf.random_normal([in_size, out_size])) 
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1) 
    Wx_plus_b = tf.matmul(inputs, Weights) + biases 
    if activation_function is None: 
        outputs = Wx_plus_b 
    else: outputs = activation_function(Wx_plus_b) 
    return outputs

In [5]:
X_train = np.vstack((xtr1, xtr2))
X_train = np.vstack((X_train, xtr3))
X_train = np.vstack((X_train, xtr4))
Y_train = np.hstack((ylab1, ylab2))
Y_train = np.hstack((Y_train, ylab3))
Y_train = np.hstack((Y_train, ylab4))
print(X_train.shape)
print(Y_train.shape)

(40000, 3072)
(40000,)


In [6]:
sess = tf.InteractiveSession()

#定义添加隐含层的函数
def add_layer(inputs, in_size, out_size,keep_prob=1.0,activation_function=None):
    Weights = tf.Variable(tf.truncated_normal([in_size,out_size],stddev=0.1))
    biases = tf.Variable(tf.zeros([out_size]))
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
        outputs = tf.nn.dropout(outputs,keep_prob)  #随机失活
    return outputs

In [11]:
# holder变量
x = tf.placeholder(tf.float32,[None,3072])
y_ = tf.placeholder(tf.float32,[None,1])
keep_prob = tf.placeholder(tf.float32)     # 概率

h1 = add_layer(x,3072,300,keep_prob,tf.nn.relu)
##输出层
w = tf.Variable(tf.zeros([300,10]))     #300*10
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(h1,w)+b)

#定义loss,optimizer
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),reduction_indices=[1]))
train_step  =tf.train.AdagradOptimizer(0.35).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))       #高维度的
acuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))    #要用reduce_mean
print(acuracy)
tf.global_variables_initializer().run()
for i in range(0,30000,100):
    batch_x = X_train[i:i+100, :]
    batch_y = Y_train[i:i+100]
    batch_y = batch_y.reshape(100, 1)
    #print(batch_x.shape)
    #print(batch_y.shape)
    
    train_step.run({x:batch_x,y_:batch_y,keep_prob:0.05})
    if i%1000==0:
        train_accuracy = acuracy.eval({x:batch_x,y_:batch_y,keep_prob:1.0})
        print("step %d,train_accuracy %f"%(i,train_accuracy))

###########test
ylab5 = ylab5.reshape(10000, 1)
print (acuracy.eval({x:xtr5,y_:ylab5,keep_prob:1.0}))

Tensor("Mean_11:0", shape=(), dtype=float32)
step 0,train_accuracy 1.000000
step 1000,train_accuracy 1.000000
step 2000,train_accuracy 1.000000
step 3000,train_accuracy 1.000000
step 4000,train_accuracy 1.000000
step 5000,train_accuracy 1.000000
step 6000,train_accuracy 1.000000
step 7000,train_accuracy 1.000000
step 8000,train_accuracy 1.000000
step 9000,train_accuracy 1.000000
step 10000,train_accuracy 1.000000
step 11000,train_accuracy 1.000000
step 12000,train_accuracy 1.000000
step 13000,train_accuracy 1.000000
step 14000,train_accuracy 1.000000
step 15000,train_accuracy 1.000000
step 16000,train_accuracy 1.000000
step 17000,train_accuracy 1.000000
step 18000,train_accuracy 1.000000
step 19000,train_accuracy 1.000000
step 20000,train_accuracy 1.000000
step 21000,train_accuracy 1.000000
step 22000,train_accuracy 1.000000
step 23000,train_accuracy 1.000000
step 24000,train_accuracy 1.000000
step 25000,train_accuracy 1.000000
step 26000,train_accuracy 1.000000
step 27000,train_accura

In [29]:
xs = tf.placeholder(tf.float32, [None, 3072]) 
ys = tf.placeholder(tf.float32, [None, 1])

# 3.定义神经层：隐藏层和预测层 
# add hidden layer 输入值是 xs，在隐藏层有 10 个神经元 
l1 = add_layer(xs, 3072, 1000, activation_function=tf.nn.relu)
# add output layer 输入值是隐藏层 l1，在预测层输出 1 个结果 
prediction = add_layer(l1, 1000, 10, activation_function=None) 


# 4.定义 loss 表达式 
# the error between prediciton and real data 
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1])) 


# 5.选择 optimizer 使 loss 达到最小 
# 这一行定义了用什么方式去减少 loss，学习率是 0.1 
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

# important step 对所有变量进行初始化 
init = tf.initialize_all_variables() 
sess = tf.Session() 
# 上面定义的都没有运算，直到 sess.run 才会开始运算 
sess.run(init)

# 迭代 1000 次学习，sess.run optimizer 
for i in range(1000): 
    # training train_step 和 loss 都是由 placeholder 定义的运算，所以这里要用 feed 传入参数 
    a = X_train[i].reshape(1, 3072)
    b = Y_train[i].reshape(1, 1)
    sess.run(train_step, feed_dict={xs: a, ys: b})
    if i % 50 == 0: 
        # to see the step improvement
        a = xtr5[i].reshape(1, 3072)
        b = ylab5[i].reshape(1, 1)
        print(sess.run(loss, feed_dict={xs: a, ys: b}))


inf
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [ ]:
# 2.定义节点准备接收数据 
# define placeholder for inputs to network 
xs = tf.placeholder(tf.float32, [None, 3074]) 
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)     # 概率

h1 = add_layer(x,784,300,keep_prob,tf.nn.relu)
##输出层
w = tf.Variable(tf.zeros([300,10]))     #300*10
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(h1,w)+b)

#定义loss,optimizer
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),reduction_indices=[1]))
train_step = tf.train.AdagradOptimizer(0.35).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))       #高维度的
acuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))    #要用reduce_mean

tf.global_variables_initializer().run()
for i in range(3000):
    batch_x,batch_y  = mnist.train.next_batch(100)
    train_step.run({x:batch_x,y_:batch_y,keep_prob:0.75})
    if i%1000==0:
        train_accuracy = acuracy.eval({x:batch_x,y_:batch_y,keep_prob:1.0})
        print("step %d,train_accuracy %g"%(i,train_accuracy))

###########test

print acuracy.eval({x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0})
运行上述程序,得到正确为:0.9784,而这仅仅是加了一个隐含层而已.
上面的隐含层的节点加的是300个神经元,如果要再加上一层400个神经元的非常的简单.

h1 = add_layer(x,784,300,keep_prob,tf.nn.relu)
h2 = add_layer(h1,300,400,keep_prob,tf.nn.relu)
##输出层
w = tf.Variable(tf.zeros([400,10]))     #300*10
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(h2,w)+b)
# 3.定义神经层：隐藏层和预测层 
# add hidden layer 输入值是 xs，在隐藏层有 10 个神经元 
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)
# add output layer 输入值是隐藏层 l1，在预测层输出 1 个结果 

In [4]:
x_data = np.linspace(-1,1,300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise
print(x_data.shape)
print(noise.shape)
print(y_data.shape)

(300, 1)
(300, 1)
(300, 1)


In [ ]:
#超参数的选择,神经网络的层数N,激活函数的选择,
for i in range(2):
    for j in range(2,6):
       
            

In [ ]:

# 2.定义节点准备接收数据 
# define placeholder for inputs to network 
xs = tf.placeholder(tf.float32, [None, 1]) 
ys = tf.placeholder(tf.float32, [None, 1])
# 3.定义神经层：隐藏层和预测层 
# add hidden layer 输入值是 xs，在隐藏层有 10 个神经元 
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)
# add output layer 输入值是隐藏层 l1，在预测层输出 1 个结果 
prediction = add_layer(l1, 10, 1, activation_function=None) 
# 4.定义 loss 表达式 
# the error between prediciton and real data 
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), reduction_indices=[1])) 
# 5.选择 optimizer 使 loss 达到最小 
# 这一行定义了用什么方式去减少 loss，学习率是 0.1 
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
# important step 对所有变量进行初始化 
init = tf.initialize_all_variables() 
sess = tf.Session() 
# 上面定义的都没有运算，直到 sess.run 才会开始运算 
sess.run(init)
# 迭代 1000 次学习，
sess.run optimizer for i in range(1000): 
    # training train_step 和 loss 都是由 placeholder 定义的运算，所以这里要用 feed 传入参数 
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data}) if i % 50 == 0: 
        # to see the step improvement 
        print(sess.run(loss, feed_dict={xs: x_data, ys: y_data}))

